### PS-s4e11 - [Predicting Loan Payback](https://www.kaggle.com/competitions/playground-series-s5e11/code?competitionId=91722&sortBy=scoreDescending&excludeNonAccessedDatasources=true)

Playground Series - Season 5, Episode 11

&nbsp;

In notebook [PS-s5e11 | first glance.1](https://www.kaggle.com/code/nina2025/ps-s5e11-first-glance-1/edit) was deleted several feat-columns one by one,
and in this notebook PS-s5e11 | first glance.2], we'll do the opposite – add several feat-columns one by one.

Passing this whole thing through the h-blend gateway. For this, the top work - [Simple XGBoost s5e11](https://www.kaggle.com/code/yousefelshahat2/simple-xgboost-s5e11) of the current moment by expert - [yousef Elshahat](https://www.kaggle.com/yousefelshahat2)

Let's see what happens.

&nbsp;

Public solutions:

||[LB](https://www.kaggle.com/competitions/playground-series-s5e11/leaderboard)| &nbsp; | | &nbsp; | &nbsp; | &nbsp; |
|:-|:-| :-: | :-: | :-: | :-: | :-: |
|  | [0.92407](https://www.kaggle.com/code/yousefelshahat2/simple-xgboost-s5e11?scriptVersionId=272522813) |&nbsp;v.6&nbsp;| original - [Simple XGBoost s5e11](https://www.kaggle.com/code/yousefelshahat2/simple-xgboost-s5e11) | expert | [yousef Elshahat](https://www.kaggle.com/yousefelshahat2) | World |
|||||||
| | | | **main weights** | **sort** | **correct weights** |
| | [0.92429](https://www.kaggle.com/code/nina2025/ps-s5e11-first-glance-2?scriptVersionId=272604654) | v.[1](#) | [ 0.20 +0.20 +0.20 +0.20 +0.20 ] |[asc/desc](#blend)|[+0,21,+0.11, -0,07,-0.10,-0.15]|
| | [0.98432](https://www.kaggle.com/code/nina2025/ps-s5e11-first-glance-2?scriptVersionId=272607416) | v.[2](#) | [ 0.40 +0.15 +0.15 +0.15 +0.15 ] |[asc/desc](#blend)|[+0,21,+0.11, -0,07,-0.10,-0.15]|
|||||||
|  | [0.92443](https://www.kaggle.com/code/nina2025/ps-s5e11-first-glance-1-feats?scriptVersionId=272824123) |&nbsp;v.4&nbsp;| original - FE &nbsp;-&nbsp; [PS-s5e11 . first glance.1 - feats](https://www.kaggle.com/code/nina2025/ps-s5e11-first-glance-1-feats) | master | [F.A.Nina](https://www.kaggle.com/code/nina2025) | Georgia |
|||||||
| | [?]() | v.[3](#) | ( original - FE &nbsp;**x**&nbsp; FE + original ) . [ 50% &nbsp;**x**&nbsp; 50% ] |[asc/desc](#blend)|[ +0,05,-0.05 ]|

In [ ]:
pip install --upgrade xgboost scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb; from xgboost import XGBRegressor

import warnings; warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/kaggle/input/playground-series-s5e11/train.csv")

print( df['gender']           .unique() )
print( df['marital_status']   .unique() )
print( df['education_level']  .unique() )
print( df['employment_status'].unique() )

In [ ]:
def fe_map(df):
    _map_gender = {
        'Female'  : 1.01, 
        'Male'    : 1.02,
        'Other'   : 1.03
    }
    _map_marital_status = {
        'Single'  : 2.011, 
        'Married' : 2.012,
        "Divorced": 2.013,
        'Widowed' : 2.014 
    }
    _map_education_level = {
        "High School": 3.11,
        "Master's"   : 3.12,
        "Bachelor's" : 3.13,
        'PhD'        : 3.14,
        'Other'      : 3.15 
    }
    _map_employment_status = {
        'Self-employed': 4.001, 
        'Employed'     : 4.002, 
        'Unemployed'   : 4.003, 
        'Retired'      : 4.004,
        'Student'      : 4.005 
    }
    df['_map_f1'] = df['gender']           .map(_map_gender)
    df['_map_f2'] = df['marital_status']   .map(_map_marital_status)
    df['_map_f3'] = df['education_level']  .map(_map_education_level)
    df['_map_f4'] = df['employment_status'].map(_map_employment_status)
    return df

def _fe_1(df):
    df['_feat_1_1'] = df['_map_f4'] * df['_map_f1']
    df['_feat_1_2'] = df['_map_f4'] * df['_map_f2']
    df['_feat_1_3'] = df['_map_f4'] * df['_map_f3']
    return df

def _fe_2(df):
    df['_feat_2_1'] = df['_map_f4'] * df['_map_f1'] * df['_map_f2']
    df['_feat_2_2'] = df['_map_f4'] * df['_map_f1'] * df['_map_f3']
    df['_feat_2_3'] = df['_map_f4'] * df['_map_f2'] * df['_map_f3']
    return df

def _fe_3(df):
    df['_feat_3_1'] = df['_map_f1'] * df['_map_f2']
    df['_feat_3_2'] = df['_map_f1'] * df['_map_f3'] 
    df['_feat_3_3'] = df['_map_f1'] * df['_map_f4']
    df['_feat_3_4'] = df['_map_f2'] * df['_map_f3']
    df['_feat_3_5'] = df['_map_f2'] * df['_map_f4']
    df['_feat_3_3'] = df['_map_f3'] * df['_map_f4']
    return df

def _fe_4(df):
    df['_feat_4_1'] = df['_map_f4'] * df['_map_f1']/(df['_map_f2'] + df['_map_f3'] +0.0000001)
    df['_feat_4_2'] = df['_map_f4'] * df['_map_f2']/(df['_map_f1'] + df['_map_f3'] +0.0000001)
    df['_feat_4_3'] = df['_map_f4'] * df['_map_f3']/(df['_map_f2'] + df['_map_f1'] +0.0000001)
    return df

def fen(df):
    df = fe_map(df)
    df =  _fe_1(df)
    df =  _fe_2(df)
    df =  _fe_3(df)
    df =  _fe_4(df)
    df.drop(columns='_map_f1', inplace=True)
    df.drop(columns='_map_f2', inplace=True)
    df.drop(columns='_map_f3', inplace=True)
    df.drop(columns='_map_f4', inplace=True)
    return df

In [ ]:
def make_subm(file_submission):

    df = pd.read_csv("/kaggle/input/playground-series-s5e11/train.csv")
    df_test = pd.read_csv("/kaggle/input/playground-series-s5e11/test.csv")
    target = df.columns.tolist()[-1]
    print(df.shape)
    df.head()

    df      = fen(df)
    df_test = fen(df_test)
    
    def create_frequency_features(train_df, test_df, cols, num, cat):
        """
        Add frequency and binning features to the dataset.
        
        - For each column, create <col>_freq = how often each value appears in train data.
        - For numeric columns, split values into 5 and 10 quantile bins (groups) to show rank or range.
        """
        train, test = train_df.copy(), test_df.copy()
    
        for col in cols:
            # Frequency encoding: how common each value is
            freq = train[col].value_counts(normalize=True)
            train[f"{col}_freq"] = train[col].map(freq)
            test[f"{col}_freq"] = test[col].map(freq).fillna(train[f"{col}_freq"].mean())
    
            # Binning: group numeric values into quantiles
            if col in num:
                for q in [5, 10, 15]:
                    try:
                        train[f"{col}_bin{q}"], bins = pd.qcut(train[col], q=q, labels=False, retbins=True, duplicates="drop")
                        test[f"{col}_bin{q}"] = pd.cut(test[col], bins=bins, labels=False, include_lowest=True)
                    except Exception:
                        train[f"{col}_bin{q}"] = test[f"{col}_bin{q}"] = 0
    
        new_num = train.drop(columns=cat+[target]).columns.tolist()
        return train, test
    
    
    # Identify feature
    cols = df.drop(columns=target).columns.tolist()
    
    # Categorical features
    cat = [col for col in cols if df[col].dtype in ["object","category"] and col != target]
    
    # Numerical features
    num = [col for col in cols if df[col].dtype not in ["object","category","bool"] and col not in ["id", target]]
    
    # Creating new features based on the frequency of numerical features
    df, df_test = create_frequency_features(df, df_test, cols, num, cat)
    
    # Preparing categorical features
    df[cat], df_test[cat] = df[cat].astype("category"), df_test[cat].astype("category")
    
    # Dropping ID and duplicates
    df.drop(columns="id", inplace=True)
    df.drop_duplicates(inplace=True)
    
    
    # print(df.columns.tolist()); df.head()
    
    
    dtrain = xgb.DMatrix(
        df.drop(columns=target),
        label=df[target],
        enable_categorical=True
    )
    
    # Define XGBoost parameters for classification
    xgb_params = {
        'tree_method': 'hist',
        'device': 'cuda',
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        'random_state': 42,
        'max_depth': 4,
        'scale_pos_weight':1,
        'min_child_weight': 89,
    }
    
    # Run cross-validation
    cv_results = xgb.cv(
        params=xgb_params,
        dtrain=dtrain,
        nfold=5,
        num_boost_round=2000,
        metrics='auc',
        verbose_eval=100,
        early_stopping_rounds=50,
    )
    
    # Display last few CV results
    print(cv_results.tail())
    
    # Extract best boosting round
    best_round = cv_results['test-auc-mean'].idxmax()
    best_auc = cv_results['test-auc-mean'][best_round]
    print(f"Best round: {best_round}, Best CV AUC: {best_auc:.7f}")
    
    
    # putting the n_estimator at the average early stopping point to avoid overfitting
    last_round = len(cv_results) - 1
    xgb_params["n_estimators"] = last_round + 10
    
    
    # Prepare training data
    X_train = df.drop(columns=target)
    y_train = df[target]
    
    # Train XGBoost model
    model = XGBRegressor(**xgb_params, enable_categorical=True)
    model.fit(X_train, y_train)

    
    if hasattr(model, "feature_importances_"):
        import seaborn as sns, matplotlib.pyplot as plt
        importances = model.feature_importances_
        importance_df = pd.DataFrame({'feature': X_train.columns,'importance': importances})
        importance_df = importance_df.sort_values('importance', ascending=False)
        plt.style.use('fivethirtyeight')
        plt.figure(figsize=(12, 20))
        sns.barplot(x='importance',y='feature', data=importance_df.head(50)) 
        plt.title('Feature Importance')
        plt.xlabel('Importance Score')
        plt.ylabel('Features')
        plt.tight_layout()
        plt.show()
    
    
    # Predict on test set
    pred = model.predict(df_test.drop(columns = "id"))
    
    # Prepare submission
    sub = pd.DataFrame({
        "id": df_test["id"],
        target: pred
    })
    
    # Save submission file
    sub.to_csv(file_submission, index=False)

In [ ]:
make_subm(f"subm_plus.csv")

In [ ]:
import os,ast,copy
import numpy as np
import pandas as pd

from bokeh.plotting import figure, gridplot 
from bokeh.io import output_file, show, output_notebook
output_notebook()

def color_scheme(dk,color):
    colors    = ['red','green','blue']
    clr_alls  = ['gold','crimson',"forestgreen",'mediumblue',"magenta"]
    clr_alls2 = ['crimson','blue']
    clr_alls3 = ['darkmagenta',"forestgreen",'mediumblue']
    clr_alls4 = ['crimson',"darkgreen",'mediumblue','brown']
    clr_alls4m= ['red',"forestgreen",'mediumblue',"darkmagenta"]
    clr_alls4j= ['red',"green",'blue',"sienna"]
    clr_alls4i= ['crimson',"green",'mediumblue',"chocolate"]
    clr_alls5 = ['red',"forestgreen",'mediumblue',"darkmagenta",'crimson']
    clr_Red   = ["firebrick","orangered","crimson",'tomato',"red"]
    clr_Red4  = ["firebrick","orangered","crimson",'red']
    clr_Red31 = ["crimson",'red','gold']
    clr_Red13 = ['gold',"crimson",'red']
    clr_Green = ["darkgreen","limegreen","green",'lime',"forestgreen"]
    clr_Green2= ['olivedrab',"darkgreen","forestgreen"]
    clr_Green3= ["darkmagenta",'olivedrab',"darkgreen"]
    clr_Green4= ["darkgreen","forestgreen","limegreen","lime"]
    clr_Blue  = ['midnightblue',"royalblue","mediumblue","blue","steelblue",'cyan']
    clr_Blue4 = ['midnightblue',"royalblue","mediumblue","deepskyblue"]
    clr_Brown = ["maroon","sienna","chocolate","sandybrown",'brown']
    clr_Brown3= ["maroon","sienna","sandybrown"]
    clr_Brown4= ["maroon","sienna","chocolate","sandybrown"]
    clr_Brown5= ["maroon","sienna","chocolate","sandybrown",'gold']
    clr_Two   = ['crimson','mediumblue']
    clr_Two2  = ['crimson','darkgreen']
    clr_tes3  = ['limegreen',"magenta",'red']
    clr_tes3b = ['darkmagenta',"magenta",'red']
    clr_tes6  = ['limegreen'] + clr_Brown
    clr_tes7  = ['limegreen'] + clr_Brown4 + ["magenta"]+["darkmagenta"]
    clr_tes8  = clr_Red4 + clr_Blue4
    clr_tes9  = clr_Red4 + ['darkmagenta'] + clr_Blue4
    clr_tes10 = clr_Brown + clr_Green
    clr_tes11 = clr_Brown + ['red','darkmagenta'] + clr_Green
    l = len(dk['subm'])
    if color == 'Two2'  : colors = clr_Two2   [0:l]
    if color == 'Two'   : colors = clr_Two    [0:l]
    if color == 'alls'  : colors = clr_alls   [0:l]
    if color == 'alls2' : colors = clr_alls2  [0:l]
    if color == 'alls3' : colors = clr_alls3  [0:l]
    if color == 'alls4' : colors = clr_alls4  [0:l]
    if color == 'alls4m': colors = clr_alls4m [0:l]
    if color == 'alls4i': colors = clr_alls4i [0:l]
    if color == 'alls4j': colors = clr_alls4j [0:l]
    if color == 'alls5' : colors = clr_alls5  [0:l]
    if color == 'red'   : colors = clr_Red    [0:l]
    if color == 'red4'  : colors = clr_Red4   [0:l]
    if color == 'red31' : colors = clr_Red31  [0:l]
    if color == 'red13' : colors = clr_Red13  [0:l]
    if color == 'green' : colors = clr_Green  [0:l]
    if color == 'green2': colors = clr_Green2 [0:l]
    if color == 'green3': colors = clr_Green3 [0:l]
    if color == 'green4': colors = clr_Green4 [0:l]
    if color == 'blue'  : colors = clr_Blue   [0:l]
    if color == 'blue4' : colors = clr_Blue4  [0:l]
    if color == 'brown' : colors = clr_Brown  [0:l]
    if color == 'brown3': colors = clr_Brown3 [0:l]
    if color == 'brown4': colors = clr_Brown4 [0:l]
    if color == 'brown5': colors = clr_Brown5 [0:l]
    if color == 'tes3'  : colors = clr_tes3   [0:l]
    if color == 'tes3b' : colors = clr_tes3b  [0:l]
    if color == 'tes6'  : colors = clr_tes6   [0:l]
    if color == 'tes7'  : colors = clr_tes7   [0:l]
    if color == 'tes8'  : colors = clr_tes8   [0:l]
    if color == 'tes9'  : colors = clr_tes9   [0:l]
    if color == 'tes10' : colors = clr_tes10  [0:l]
    if color == 'tes11' : colors = clr_tes11  [0:l]
    return colors


def bokeh_show(
        params,
        df_cross,
        colors, 
        show_figures1, 
        show_figures2, wps_fig2,
        color_cross):
    
    def dossier(js,subms,cols):
        def quant(i,js,subms,cols):
            return {"c" : i, "q" : sum([1 for subm in cols[i] if subm == subms[js]])}
        return {
            'name' : subms[js],
            'q_in' : [quant(i,js,subms,cols) for i in range(len(subms))]
        }
    alls = pd.read_csv(f'tida_desc.csv')
    matrix = [ast.literal_eval(str(row.alls)) for row in alls.itertuples()]
    subms = sorted(matrix[0])
    cols = [[data[i] for data in matrix] for i in range(len(subms))]
    df_subms = pd.DataFrame({f'col_{i}': [x[i] for x in matrix] for i in range(len(subms))})
    dossiers = [dossier(js,subms,cols) for js in range(len(subms))]
    subm_names = [one_dossier['name'] for one_dossier in dossiers]
    figures1,qss,i = [],[],0
    height = 100 if len(colors)==2\
        else 134 if len(colors)==3 else (154 if len(colors)==4 else 174)
    for one_dossier in dossiers: 
        i_col = 'alls. ' + str(one_dossier['q_in'][i]['c'])
        qs = [one['q'] for one in one_dossier['q_in']]
        x_names = [name.replace("Group","").replace("subm_","") for name in subm_names]
        width = 157  if len(colors) == 5\
            else (121 if len(colors) == 8\
            else (131 if len(colors) == 9\
            else (141 if len(colors) == 10\
            else (171 if len(colors) == 11 else 100))))
        f = figure(x_range=x_names,width=width, height=height, title=i_col)
        f.vbar(x=x_names, width=0.585, top=qs, color=colors)
        figures1.append(f)
        qss.append(qs)
        i+=1
    grid = gridplot([figures1])
    output_file('tida_alls.html')
    if show_figures1 == True: show(grid)
    sub_wts = params['subwts']
    main_wts = [subm['weight'] for subm in params['subm']]
    mms,acc_mass = [],[]
    for j in range(len(dossiers)):
        one_dossier = dossiers[j]
        qs = [one['q'] for one in one_dossier['q_in']]
        mm = [qs[h] * (main_wts[j] + sub_wts[h]) for h in range(len(sub_wts))]
        mass = sum(mm)
        mms.append(mm)
        acc_mass.append(round(mass))                        #subm_names[::-1]
    y_names = [name + " - " + str(mass) for name,mass in zip(subm_names,acc_mass)]
    f1 = figure(y_range=y_names, width=313, height=height, title='relations of general masses')
    f1.hbar(y=y_names, height=0.585, right=acc_mass, left=0, color=colors)
    output_file('tida_alls2.html')
    alls = [f'alls.{i}' for i in range(len(dossiers))]
    subm = [f'sub{i}'   for i in range(len(dossiers))] 
    mmsT  = np.asarray(mms).T
    data = {'cols' : alls}
    for i in range(len(dossiers)): data[f'sub{i}'] = mmsT[i,:]
    f2 = figure(y_range=alls, height=height, width=274, title=" ( relations of columns masses )")
    f2.hbar_stack(subm, y='cols', height=0.585, color=colors, source=data)
    qssT  = np.asarray(qss).T
    data = {'cols' : alls}
    for i in range(len(dossiers)): data[f'sub{i}'] = qssT[i,:]
    f3 = figure(y_range=alls, height=height, width=215, title="ratios in columns")
    f3.hbar_stack(subm, y='cols', height=0.585, color=colors, source=data)
    grid = gridplot([[f3,f2,f1]])
    show(grid)
    if show_figures2 == True:
        def read(params,i):
            FiN = params["path"] + params["subm"][i]["name"] + ".csv"
            target_name_back = {'target':params["target"],'pred':params["target"]}
            return pd.read_csv(FiN).rename(columns=target_name_back)
        dfs = [read(params,i) for i in range(len(params["subm"]))] + [df_cross]
        f   = figure(width=800, height=274)
        f.title.text = 'Click on legend entries to mute the corresponding lines'
        b,e        = 21000,21121
        line_x     = [dfs[i][b:e]['id']            for i in range(len(dfs))]
        line_y     = [dfs[i][b:e]['loan_paid_back'] for i in range(len(dfs))]
        color      = colors + [color_cross]
        alpha      = [0.8 for i in range(len(dfs)-1)] + [0.95]
        lws        = [1.0 for i in range(len(dfs)-1)] + [1.00]
        legend = subm_names + ['cross']
        for i in range(len(legend)):
            f.line(line_x[i], line_y[i], line_width=lws[i], color=color[i], alpha=alpha[i],
                   muted_color='white',legend_label=legend[i])
        f.legend.location = "top_left"
        f.legend.click_policy="mute"
        show(f)


def h_blend(params,color,cross='silver',
            figures1=False,figures2=False,wf2=555,
            details=False):

    color_cross = cross

    dk = copy.deepcopy(params)

    show_details,show_figures1,show_figures2 = details,figures1,figures2

    file_short_names = [subm['name'] for subm in params['subm']]
    type_sort    = params['type_sort'][0]
    dk['asc']    = params['type_sort'][1]
    dk['desc']   = params['type_sort'][2]
    dk['id']     = params['id_target'][0]
    dk['target'] = params['id_target'][1]
# ------------------------------------------------------------------------
    def read(dk,i):
        tnm = dk["subm"][i]["name"]
        FiN = dk["path"] + tnm + ".csv"
        return pd.read_csv(FiN).rename(columns={
            'target':tnm, 'pred':tnm, dk["target"]:tnm})
        
    def merge(dfs_subm):
        df_subms = pd.merge(dfs_subm[0],  dfs_subm[1], on=[dk['id']])
        for i in range(2, len(dk["subm"])): 
            df_subms = pd.merge(df_subms, dfs_subm[i], on=[dk['id']])
        return df_subms
        
    def da(dk,sorting_direction,show_details):
        
        df_subms = merge([read(dk,i) for i in range(len(dk["subm"]))])
        cols = [col for col in df_subms.columns if col != dk['id']]
        short_name_cols = [c for c in cols]
        
        def alls1(x, sd=sorting_direction,cs=cols):
            reverse = True if sd=='desc' else False
            tes = {c: x[c] for c in cs}.items()
            subms_sorted = [t[0] for t in sorted(tes,key=lambda k:k[1],reverse=reverse)]
            return subms_sorted

        import random

        def alls2(x, sd=sorting_direction,cs=cols):
            reverse = True if sd=='desc' else False
            tes = {c: x[c] for c in cs}.items()
            subms_random = [t[0] for t in tes]
            random.shuffle(subms_random)
            return subms_random

        alls = alls1 if type_sort == 'asc/desc' else alls2
            
        def summa(x,cs,wts,ic_alls): 
            return sum([x[cs[j]] * (wts[0][j] + wts[1][ic_alls[j]]) for j in range(len(cs))])
            
        wts = [[[e['weight'] for e in dk["subm"]], [w for w in dk["subwts" ]]]]
          
        def correct(x, cs=cols, wts=wts):
            i = [x['alls'].index(c) for c in short_name_cols]
            return summa(x,cs,wts[0],i)

        if len(wts) == 1:
            correct_sub_weights = [wt for wt in dk["subwts"]]
            weights = [subm['weight'] for subm in dk["subm"]]
            def correct(x, cs=cols, w=weights, cw=correct_sub_weights):
                ic = [x['alls'].index(c) for c in short_name_cols]
                cS = [x[cols[j]] * (w[j] + cw[ic[j]]) for j in range(len(cols))]
                return sum(cS)
                   
        def amxm(x, cs=cols):
            list_values = x[cs].to_list()
            mxm = abs(max(list_values)-min(list_values))
            return mxm

        if len(wts) > 1:
            df_subms['mx-m']   = df_subms.apply(lambda x: amxm   (x), axis=1)
        df_subms['alls']       = df_subms.apply(lambda x: alls   (x), axis=1)
        df_subms[dk["target"]] = df_subms.apply(lambda x: correct(x), axis=1)
        schema_rename = { old_nc:new_shnc for old_nc, new_shnc in zip(cols, short_name_cols) }
        df_subms = df_subms.rename(columns=schema_rename)
        df_subms = df_subms.rename(columns={dk["target"]:"ensemble"})
        df_subms.insert(loc=1, column=' _ ', value=['   '] * len(df_subms))
        df_subms[' _ '] = df_subms[' _ '].astype(str)
        pd.set_option('display.max_rows',100)
        pd.set_option('display.float_format', '{:.5f}'.format)
        vcols = [dk['id']]+[' _ '] + short_name_cols + [' _ ']+['alls']+[' _ ']+['ensemble']
        if len(wts) > 1: vcols.append([' _ '] + ['mx-m'])
        df_subms = df_subms[vcols]
        if show_details: display(df_subms.head(5))
        pd.set_option('display.float_format', '{:.5f}'.format)
        df_subms = df_subms.rename(columns={"ensemble":dk["target"]})
        df_subms.to_csv(f'tida_{sorting_direction}.csv', index=False)
        return df_subms[[dk['id'],dk['target']]]
   
    def ensemble_da(dk,        show_details): 
        dfD    = da(dk,'desc', show_details)
        dfA    = da(dk,'asc',  show_details)
        dfA[dk['target']] = dk['desc']*dfD[dk['target']] + dfA[dk['target']]*dk['asc']
        return dfA

    da = ensemble_da(dk,show_details)
    colors = color_scheme(dk, color)
    bokeh_show(dk, da, colors, show_figures1, show_figures2, wf2, color_cross)
    return  da


def matrix_vs(path,fs_names):
    def load(path,fs_names):
        dfs = [pd.read_csv(path + name_subm +'.csv') for name_subm in fs_names]
        for i in range(len(dfs)):
            dfs[i] = dfs[i].rename(columns={"loan_paid_back": f'{fs_names[i]}'})
        dfsm = pd.merge(dfs[0], dfs[1], on="id")
        for i in range(2,len(dfs)):
            dfsm = pd.merge(dfsm,dfs[i],on='id')
        return dfsm   
    def make_list_vs(fs_names):
        list = []
        for i in range(0,len(fs_names)-1):
            for j in range(i+1,len(fs_names)):
                list.append(fs_names[i] + "_vs_" + fs_names[j])
        return list
    def get_mvs(dfs, list_vs):
        def get_abs_distance(x,t1,t2):
            return abs(x[t1]-x[t2])
        for vs in list_vs:
            t = vs.split('_vs_')
            dfs[vs] = dfs.apply(lambda x: get_abs_distance(x,t[0],t[1]), axis=1)
        return dfs   
    def distance_vs(name, st_names, list_vs, dfs):
        distances = []
        for st in st_names:
            vs_between = name + "_vs_" + st
            if vs_between not in list_vs:
                distances.append(0)
            else: distances.append(round(dfs[vs_between].sum()))
        return distances
    dfs = load(path,fs_names)
    list_vs = make_list_vs(fs_names)
    mvs = get_mvs(dfs, list_vs)
    m1 = pd.DataFrame({'subm':fs_names})
    m2 = pd.DataFrame({ name :distance_vs(name, fs_names, list_vs, mvs) for name in fs_names})
    matrix = pd.concat([m1,m2],axis=1)
    return matrix


def display_distances(params):
    files = [subm['name'] for subm in params['subm']]
    distances = matrix_vs ( params['path'], files )            
    display(distances)

In [ ]:
# subm_minus.LB is already known = 0.92443
df = pd.read_csv('/kaggle/input/ps-s5e11-first-glance-1-feats/submission.csv')
df . to_csv('subm_minus.csv',index=False)

path = f'/kaggle/working/subm_'

params = {
      'path'     : path,            
      'id_target': ['id',"loan_paid_back"],          
      'type_sort': ['asc/desc', 0.30,0.70],
      'subwts'   : [ +0.05,-0.05 ],
      'subm'     : [
         { 'name': f'minus','weight':+0.50 },
         { 'name': f'plus', 'weight':+0.50 },]
}

df_cross = h_blend(params,color='alls2',figures1=True,figures2=True,details=True)

display_distances(params)

## Submit

In [ ]:
df_cross . to_csv('submission.csv',index=False)

df_cross